<a href="https://colab.research.google.com/github/cboyda/LighthouseLabs/blob/main/Project_Stats_model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merge Data

In [1]:
import pandas as pd

In [2]:
# how to mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_foursquare = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Project-Statistics_city_bikes_FourSquare.csv')

In [4]:
df_yelp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/Project-Statistics_city_bikes_yelp.csv')

In [5]:
merged_df = df_foursquare.merge(df_yelp, on=['city', 'station_name','latitude', 'longitude','empty_slots','slots','free_bikes','ebikes'])

In [6]:
df = merged_df.copy()

In [7]:
df.shape

(242, 12)

In [8]:
df

,city,station_name,empty_slots,slots,free_bikes,ebikes,latitude,longitude,station_location,location_count,yelp_location_count,yelp_review_count
0,Vancouver,10th & Cambie,22,35,13,4,49.262487,-123.114397,"49.262487,-123.114397",0,8,42
1,Vancouver,Yaletown-Roundhouse Station,6,16,10,0,49.274566,-123.121817,"49.274566,-123.121817",0,14,94
2,Vancouver,Dunsmuir & Beatty,23,26,3,1,49.279764,-123.110154,"49.279764,-123.110154",0,10,61
3,Vancouver,12th & Yukon (City Hall),14,16,2,2,49.260599,-123.113504,"49.260599,-123.113504",0,10,49
4,Vancouver,8th & Ash,15,16,1,0,49.264215,-123.117772,"49.264215,-123.117772",0,9,69
...,...,...,...,...,...,...,...,...,...,...,...,...
237,Vancouver,Burrard & 14th,12,18,4,4,49.259469,-123.145718,"49.259469,-123.145718",0,8,15
238,Vancouver,Hornby & Drake,19,24,5,2,49.277178,-123.130000,"49.277178,-123.13",0,13,176
239,Vancouver,Cardero & Bayshore,2,20,16,1,49.291597,-123.129158,"49.291597,-123.129158",0,12,1251
240,Vancouver,27th & Main,13,22,9,7,49.247204,-123.101549,"49.247204,-123.101549",0,6,173


# Build a regression model.

In [9]:
df.dtypes

city                    object
station_name            object
empty_slots              int64
slots                    int64
free_bikes               int64
ebikes                   int64
latitude               float64
longitude              float64
station_location        object
location_count           int64
yelp_location_count      int64
yelp_review_count        int64
dtype: object

## Try Ordinary Least Squares (OLS) Regression model

In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Split the dataset into input features (X) and target variable (y)
# X = df[df.columns[~df.columns.isin(['ebikes'])]] # fails since they must be numerical
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('ebikes')
X = df[numeric_cols]
y = df['ebikes']

# Add a constant column to the input features
X_const = sm.add_constant(X)

# Fit the OLS regression model
model = sm.OLS(y, X_const)
resultsOLS = model.fit()

Provide model output and an interpretation of the results. 

In [12]:
# Print the model summary
print(resultsOLS.summary())

                            OLS Regression Results                            
Dep. Variable:                 ebikes   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.168
Method:                 Least Squares   F-statistic:                     7.084
Date:                Mon, 05 Jun 2023   Prob (F-statistic):           2.23e-08
Time:                        04:16:25   Log-Likelihood:                -432.32
No. Observations:                 242   AIC:                             882.6
Df Residuals:                     233   BIC:                             914.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1690.9419    

Based on p values, only the latitude is less than 0.05 which indicates it is statistically significant.  We could try dropping all other columns and rerunning the model.

In [14]:
resultsOLS.params

const                  1690.941861
empty_slots              -0.124956
slots                     0.149090
free_bikes               -0.051673
latitude                -47.280737
longitude                -5.191709
location_count           -0.168680
yelp_location_count      -0.024606
yelp_review_count         0.000215
dtype: float64

Let's interpret the coefficients for the given example:

* empty_slots: For each unit decrease in the number of empty slots, the predicted value of the response variable (ebikes) decreases by approximately 0.125.

* slots: For each unit increase in the total number of slots, the predicted value of the response variable increases by approximately 0.149.

* free_bikes: For each unit decrease in the number of free bikes, the predicted value of the response variable decreases by approximately 0.052.

* latitude: For each unit decrease in latitude, the predicted value of the response variable decreases by approximately 47.281.

* longitude: For each unit decrease in longitude, the predicted value of the response variable decreases by approximately 5.192.

* location_count: For each unit decrease in the location count, the predicted value of the response variable decreases by approximately 0.169.

* yelp_location_count: For each unit decrease in the Yelp location count, the predicted value of the response variable decreases by approximately 0.025.

* yelp_review_count: For each unit increase in the Yelp review count, the predicted value of the response variable increases by approximately 0.0002.

In [15]:
resultsOLS.params.to_frame(name='Coefficient')

,Coefficient
const,1690.941861
empty_slots,-0.124956
slots,0.149090
free_bikes,-0.051673
latitude,-47.280737
longitude,-5.191709
location_count,-0.168680
yelp_location_count,-0.024606
yelp_review_count,0.000215


## Try with Linear Regression Model
using list method (backward variable selection)

In [19]:
# use this List comprehension method instead of manual for loops
Models = [sm.OLS(y, X[[x]]) for x in X]  # list of models
ResultsLR = [model.fit() for model in Models]  # list of results
Adj_Rsquared = [result.rsquared_adj for result in ResultsLR]  # list of adjusted R-squared
Pval = [result.pvalues for result in ResultsLR]  # list of p-values
Params = [result.params for result in ResultsLR]  # list of parameters

In [20]:
ResultsLR

In [37]:
# how many models?
len(ResultsLR) 

8

Which model do we use?  Need to find the best model, as defined by Adj. R-squared.

In [39]:
max_adj_r_squared = float('-inf')  # Initialize with a very small value

best_model_index = -1  # Variable to store the index of the best model

for i, result in enumerate(ResultsLR):
    adj_r_squared = result.rsquared_adj
    if adj_r_squared > max_adj_r_squared:
        max_adj_r_squared = adj_r_squared
        best_model_index = i

print("Best model index:", best_model_index, "of",len(ResultsLR) ,"models.")
print("Highest adjusted R-squared:", max_adj_r_squared)
print("Summary of the best model:")
print(ResultsLR[best_model_index].summary())

Best model index: 1 of 8 models.
Highest adjusted R-squared: 0.4335386978489718
Summary of the best model:
                                 OLS Regression Results                                
Dep. Variable:                 ebikes   R-squared (uncentered):                   0.436
Model:                            OLS   Adj. R-squared (uncentered):              0.434
Method:                 Least Squares   F-statistic:                              186.2
Date:                Mon, 05 Jun 2023   Prob (F-statistic):                    8.49e-32
Time:                        04:36:11   Log-Likelihood:                         -454.20
No. Observations:                 242   AIC:                                      910.4
Df Residuals:                     241   BIC:                                      913.9
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
             

In this case, the R-squared and Adjusted R-squared values are relatively low, suggesting that the independent variables may not have a strong linear relationship with the dependent variable or that there may be other important variables missing from the model. It is important to interpret these values in the context of the specific data and the research question at hand.

## Try with Generalized Linear Model (GLM)

In [22]:
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families import Gaussian

# Add a constant column to the input features
X_const = sm.add_constant(X)

# Perform linear regression using GLM with Gaussian family
model = GLM(y, X_const, family=Gaussian())
resultsGLM = model.fit()

# Get the regression coefficients
coefficients = resultsGLM.params

# Get the summary of the regression results
summary = resultsGLM.summary()

# Print the coefficients and summary
print(coefficients)
print(summary)

const                  1690.941861
empty_slots              -0.124956
slots                     0.149090
free_bikes               -0.051673
latitude                -47.280737
longitude                -5.191709
location_count           -0.168680
yelp_location_count      -0.024606
yelp_review_count         0.000215
dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:                 ebikes   No. Observations:                  242
Model:                            GLM   Df Residuals:                      233
Model Family:                Gaussian   Df Model:                            8
Link Function:               identity   Scale:                          2.1660
Method:                          IRLS   Log-Likelihood:                -432.32
Date:                Mon, 05 Jun 2023   Deviance:                       504.68
Time:                        04:21:18   Pearson chi2:                     505.
No. Iterations:                     3 

No new information from this model.

## Model Predictions

In [32]:
ResultsLR

In [57]:
y_predOLS = resultsOLS.predict(X_const)
y_predGLM = resultsGLM.predict(X_const)
#y_predLR = ResultsLR.predict(X_const)
y_predLR = ResultsLR[best_model_index].fittedvalues # since there is a list in ResultLR

In [58]:
# how accurate are these predictions?
from sklearn.metrics import r2_score
r2OLS = r2_score(y, y_predOLS) * 100
r2_formatted = "{:.2f}%".format(r2OLS) 
print("The R-squared (R2) score of the Prediction for OLS regression model is: {}".format(r2_formatted))

r2LR = r2_score(y, y_predLR) * 100
r2_formatted = "{:.2f}%".format(r2LR) 
print("The R-squared (R2) score of the Prediction for the Linear regression model is: {}".format(r2_formatted))

r2GLM = r2_score(y, y_predGLM) * 100
r2_formatted = "{:.2f}%".format(r2GLM) 
print("The R-squared (R2) score of the Prediction for the GLM regression model is: {}".format(r2_formatted))


The R-squared (R2) score of the Prediction for OLS regression model is: 19.56%
The R-squared (R2) score of the Prediction for the Linear regression model is: 3.62%
The R-squared (R2) score of the Prediction for the GLM regression model is: 19.56%


Conclusion: This tells us counting the number of nearby "parks" is not a very accurate method for predicting the number of ebikes.

## Compare Predictions to Average as Baseline

In [43]:
# Calculate the average value of the response variable
y_mean = np.mean(y)

# Create an array of predicted values with the average value repeated for all observations
y_pred_baseline = np.full_like(y, y_mean)

# Calculate the accuracy of the baseline model
baseline_accuracy = np.mean(y == y_pred_baseline) * 100

print("Baseline prediction accuracy: {:.2f}%".format(baseline_accuracy))

Baseline prediction accuracy: 23.97%


This shows us just using average would be BETTER accuracy then our model.

# Stretch

## How can you turn the regression model into a classification model?

To turn a regression model into a classification model, you can apply a threshold to the predicted values and convert them into discrete classes

### Preprocessing for Binary Value to Predict

In [47]:
# Set the threshold for classification
threshold = 0.5

# Apply the threshold to the predicted values
y_pred_class = np.where(y_predOLS >= threshold, 1, 0)

# Print the predicted class labels
print(y_pred_class)

[1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1]


These binary values for  y_pred_class could then be feed into a machine learning classification ML model.

### Try with Logit (Logistical Regression)

Logit only works when y is binary!

In [48]:
# Convert y to binary format
y_binary = np.where(y > 0, 1, 0)

In [51]:
X_const = sm.add_constant(X) #adds a column of 1's so the model will contain an intercept

model = sm.Logit(y_binary,X_const)
resultsLog = model.fit()
print(resultsLog.summary())

Optimization terminated successfully.
         Current function value: 0.631347
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  242
Model:                          Logit   Df Residuals:                      233
Method:                           MLE   Df Model:                            8
Date:                Mon, 05 Jun 2023   Pseudo R-squ.:                 0.06237
Time:                        05:18:34   Log-Likelihood:                -152.79
converged:                       True   LL-Null:                       -162.95
Covariance Type:            nonrobust   LLR p-value:                  0.009166
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                1849.4328    947.993      1.951      0.051      -8.599    3707.465
em

* The p-value for the intercept (const) is 0.051. It is slightly higher than the significance level of 0.05, indicating that the intercept's relationship with the log odds of the positive class is borderline statistically significant.
* The p-values for empty_slots, slots, free_bikes, longitude, location_count, yelp_location_count, and yelp_review_count are all greater than 0.05. These variables do not show statistical significance in predicting ebikes

The pseudo R-squared value in logistic regression measures the goodness-of-fit of the model. It indicates the proportion of the total variation in the dependent variable that is explained by the model. In your case, the pseudo R-squared value of 0.06 suggests that the model explains only a small portion of the variation in the data.

In [56]:
from sklearn.metrics import accuracy_score

# Assuming y_test contains the actual class labels and y_predLog contains the predicted class probabilities or labels
y_pred_class = np.where(y_predLog >= threshold, 1, 0)  # Apply threshold if needed to convert probabilities to class labels

accuracy = accuracy_score(y, y_pred_class)
accuracy_formatted = "{:.2f}%".format(accuracy * 100)
print("The Logistic Regression (Classification) model pridiction accuracy: {}".format(accuracy_formatted))



The Logistic Regression (Classification) model pridiction accuracy: 31.82%


The accuracy score represents the percentage of correctly predicted labels compared to the actual labels.